# This notebook serves as walkthrough for planning an experiment for creation through the OT2.
### The following modules are used and should be in the same directory as this notebook: 
* **CreateSamples** is responsible for calculating sample information, which includes component weight fractions and stock volumes
* **OT2Commands** is responsible for setting up information to be interpretted and executed by opentrons **BASED ON THE LATEST API 2.3 and above**

In [1]:
import os
import warnings
from Plan import CreateSamples
from Prepare import OT2Commands as ALH
from opentrons import simulate, execute, protocol_api
import importlib
import pandas as pd
from datetime import datetime
from pytz import timezone 

Loading files such as the labware locations in the OT2 robot. 

In [2]:
path = r"Protocol.csv"
plan = CreateSamples.get_experiment_plan(path)

In [3]:
stock_volumes = CreateSamples.concentration_from_csv('Volumes/Volumes_5.csv') # This is the .csv file that contains volume information 
stock_volumes = stock_volumes.loc[:, ~stock_volumes.columns.str.contains('^Unnamed')]
stock_volumes['Water-stock'].sum(axis=0)
stock_volumes.astype(int)

,2mM_PVP-stock,Water-stock,2mM_TA-stock,2mM_AA-stock,2mM_SN-stock,0.025mM_Seeds-stock
0,35,177,32,22,16,43
1,18,180,38,11,30,48
2,30,174,23,11,46,41
3,22,232,4,1,28,38
4,23,212,11,11,33,35
5,32,158,29,7,41,58
6,40,130,35,28,38,54
7,36,197,20,12,31,29
8,36,148,25,6,57,53
9,32,179,28,24,6,56


In [4]:
stock_volumes.sum(axis=0)

2mM_PVP-stock          1352.0
Water-stock            9061.0
2mM_TA-stock            938.0
2mM_AA-stock            734.0
2mM_SN-stock           1534.0
0.025mM_Seeds-stock    1981.0
dtype: float64

## Simulate

Run the following cells to simulate the robot protocol. 
If you look at the last cell in this section, you can see the actual steps the robot will follow to make the samples.

In [5]:
# Simulating 
labware_dir_path = r"Custom Labware"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)
protocol = simulate.get_protocol_api('2.7', extra_labware = custom_labware_dict) #

/home/huatthart/.opentrons/robot_settings.json not found. Loading defaults
/home/huatthart/.opentrons/deck_calibration.json not found. Loading defaults


In [6]:
loaded_dict = ALH.loading_labware(protocol, plan)
max_source_vol = 17350
stock_position_info = ALH.stock_well_ranges(stock_volumes, loaded_dict, max_source_vol) 

# let's print the stock position info and make sure everything is in the right postion. 
# You can also double-check that you have enought stock to make all your samples
# (i.e. only a single range of wells is provided for each stock)

In [7]:
stock_position_info

{'2mM_PVP-stock': {'Ranges': [[0, 48]],
  'Stock Wells': [A1 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 'Water-stock': {'Ranges': [[0, 48]],
  'Stock Wells': [A2 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 '2mM_TA-stock': {'Ranges': [[0, 48]],
  'Stock Wells': [A3 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 '2mM_AA-stock': {'Ranges': [[0, 48]],
  'Stock Wells': [A4 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 '2mM_SN-stock': {'Ranges': [[0, 48]],
  'Stock Wells': [B1 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 '0.025mM_Seeds-stock': {'Ranges': [[0, 48]],
  'Stock Wells': [B2 of 20mLscintillation 12 Well Plate 18000 µL on 1]}}

In [8]:
Start_pos = 0

In [9]:
directions = ALH.create_sample_making_directions(stock_volumes, stock_position_info, loaded_dict, start_position=Start_pos)
ALH.pipette_volumes_component_wise(protocol, directions, loaded_dict)
# The above command will fill the wells component wise- this means that all the first stock will be dispendes in the wells that require it. 
# If you want to fill the wells individually use the following line instead:
# ALH.pipette_volumes_sample_wise(protocol,directions, loaded_dict)

Picking up tip from A1 of Opentrons 96 Tip Rack 20 µL on 4
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 7
Aspirating 35.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Air gap
Aspirating 20.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Delaying for 0 minutes and 13.0 seconds
Touching tip
Dispensing 35.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at A1 of Corning 96 Well Plate 360 µL Flat on 6
Aspirating 18.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Air gap
Aspirating 2.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Delaying for 0 minutes and 13.0 seconds
Touching tip
Dispensing 18.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at A2 of Corning 96 Well Plate 360 µL Flat on 6
Aspirating 30.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Air gap
As

Dispensing 20.0 uL into A4 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Blowing out at A4 of Agilent 6 Reservoir 47000 µL on 5
Aspirating 16.0 uL from A3 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Air gap
Aspirating 4.0 uL from A3 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 16.0 uL into C9 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at C9 of Corning 96 Well Plate 360 µL Flat on 6
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from C9 of Corning 96 Well Plate 360 µL Flat on 6 at 100.0 uL/sec
Dispensing 20.0 uL into C9 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 20.0 uL from C9 of Corning 96 Well Plate 360 µL Flat on 6 at 100.0 uL/sec
Dispensing 20.0 uL into C9 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from A6 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A6 o

Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from A5 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A5 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Aspirating 20.0 uL from A5 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A5 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Blowing out at A5 of Agilent 6 Reservoir 47000 µL on 5
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from A4 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A4 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Aspirating 20.0 uL from A4 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A4 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Blowing out at A4 of Agilent 6 Reservoir 47000 µL on 5
Aspirating 30.0 uL from A4 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Air gap
Aspirating 20.0 uL from A4 of 20mLscintillation 12 Well Plate 1800

Blowing out at A6 of Agilent 6 Reservoir 47000 µL on 5
Mixing 2 times with a volume of 100.0 ul
Aspirating 100.0 uL from A5 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A5 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Aspirating 100.0 uL from A5 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A5 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Blowing out at A5 of Agilent 6 Reservoir 47000 µL on 5
Mixing 2 times with a volume of 100.0 ul
Aspirating 100.0 uL from A4 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A4 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Aspirating 100.0 uL from A4 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A4 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Blowing out at A4 of Agilent 6 Reservoir 47000 µL on 5
Aspirating 26.0 uL from B2 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Air gap
Aspir

Dispensing 33.0 uL into D8 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at D8 of Corning 96 Well Plate 360 µL Flat on 6
Mixing 2 times with a volume of 100.0 ul
Aspirating 100.0 uL from D8 of Corning 96 Well Plate 360 µL Flat on 6 at 200.0 uL/sec
Dispensing 100.0 uL into D8 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 100.0 uL from D8 of Corning 96 Well Plate 360 µL Flat on 6 at 200.0 uL/sec
Dispensing 100.0 uL into D8 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Mixing 2 times with a volume of 100.0 ul
Aspirating 100.0 uL from A6 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A6 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Aspirating 100.0 uL from A6 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A6 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Blowing out at A6 of Agilent 6 Reservoir 47000 µL on 5
Mixing 2 times with a volume of 100.0 ul
Aspiratin

## Execute


Run the cells below to actually execute the protocol using the robot.

Make sure that all stock solution are **not capped** and that the labware is in the correct position. 

In [26]:
Pacific = timezone('US/Pacific')
p_time = datetime.now(Pacific)
print(p_time.strftime('%H:%M:%S'))

12:06:32


In [27]:
# Executing 
protocol = execute.get_protocol_api('2.7', extra_labware = custom_labware_dict)

In [28]:
loaded_dict = ALH.loading_labware(protocol, plan)
stock_position_info = ALH.stock_well_ranges(stock_volumes, loaded_dict, max_source_vol) 

# let's print the stock position info and make sure everything is in the right postion. 
# You can also double-check that you have enought stock to make all your samples
# (i.e. only a single range of wells is provided for each stock)

stock_position_info

{'2mM_PVP-stock': {'Ranges': [[0, 48]],
  'Stock Wells': [A1 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 'Water-stock': {'Ranges': [[0, 48]],
  'Stock Wells': [A2 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 '2mM_TA-stock': {'Ranges': [[0, 48]],
  'Stock Wells': [A3 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 '2mM_AA-stock': {'Ranges': [[0, 48]],
  'Stock Wells': [A4 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 '2mM_SN-stock': {'Ranges': [[0, 48]],
  'Stock Wells': [B1 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 '2mM_Seeds-stock': {'Ranges': [[0, 48]],
  'Stock Wells': [B2 of 20mLscintillation 12 Well Plate 18000 µL on 1]}}

In [29]:

# Let's generate a dictionary containing the directions to make each sample. 
# the following line generate a dictionary where the keys corresponds to the well number ( or the index fo the dataframe of sample composition). 
# The value of each key represents the components' make up of that specific sample. 
directions = ALH.create_sample_making_directions(stock_volumes, stock_position_info, loaded_dict, start_position=Start_pos)
directions

{0: {'2mM_PVP-stock': {'Stock Position': A1 of 20mLscintillation 12 Well Plate 18000 µL on 1,
   'Destination Well Position': E1 of Corning 96 Well Plate 360 µL Flat on 6,
   'Stock Volume': 35.0},
  'Water-stock': {'Stock Position': A2 of 20mLscintillation 12 Well Plate 18000 µL on 1,
   'Destination Well Position': E1 of Corning 96 Well Plate 360 µL Flat on 6,
   'Stock Volume': 177.0},
  '2mM_TA-stock': {'Stock Position': A3 of 20mLscintillation 12 Well Plate 18000 µL on 1,
   'Destination Well Position': E1 of Corning 96 Well Plate 360 µL Flat on 6,
   'Stock Volume': 32.0},
  '2mM_AA-stock': {'Stock Position': A4 of 20mLscintillation 12 Well Plate 18000 µL on 1,
   'Destination Well Position': E1 of Corning 96 Well Plate 360 µL Flat on 6,
   'Stock Volume': 22.0},
  '2mM_SN-stock': {'Stock Position': B1 of 20mLscintillation 12 Well Plate 18000 µL on 1,
   'Destination Well Position': E1 of Corning 96 Well Plate 360 µL Flat on 6,
   'Stock Volume': 16.0},
  '2mM_Seeds-stock': {'Sto

### the following command will actully start the experiment. Once again make sure that everything is where it is supposed to be!

In [30]:
ALH.pipette_volumes_component_wise(protocol, directions, loaded_dict)

Picking up tip from A1 of Opentrons 96 Tip Rack 20 µL on 4
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 7
Aspirating 35.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Air gap
Aspirating 20.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Delaying for 0 minutes and 13.0 seconds
Touching tip
Dispensing 35.0 uL into E1 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at E1 of Corning 96 Well Plate 360 µL Flat on 6
Aspirating 18.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Air gap
Aspirating 2.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Delaying for 0 minutes and 13.0 seconds
Touching tip
Dispensing 18.0 uL into E2 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at E2 of Corning 96 Well Plate 360 µL Flat on 6
Aspirating 30.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Air gap
As

Aspirating 192.0 uL from A2 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Air gap
Aspirating 20.0 uL from A2 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Dispensing 192.0 uL into H10 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at H10 of Corning 96 Well Plate 360 µL Flat on 6
Aspirating 195.0 uL from A2 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Air gap
Aspirating 20.0 uL from A2 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Dispensing 195.0 uL into H11 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at H11 of Corning 96 Well Plate 360 µL Flat on 6
Aspirating 232.0 uL from A2 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Air gap
Aspirating 20.0 uL from A2 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Dispensing 232.0 uL into H12 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at H12 of Corning 96 Well 

Aspirating 20.0 uL from F6 of Corning 96 Well Plate 360 µL Flat on 6 at 100.0 uL/sec
Dispensing 20.0 uL into F6 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 20.0 uL from F6 of Corning 96 Well Plate 360 µL Flat on 6 at 100.0 uL/sec
Dispensing 20.0 uL into F6 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from A6 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A6 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Aspirating 20.0 uL from A6 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A6 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Blowing out at A6 of Agilent 6 Reservoir 47000 µL on 5
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from A5 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A5 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Aspirating 20.0 uL from A5 of Agilent 6 Re

Blowing out at A4 of Agilent 6 Reservoir 47000 µL on 5
Aspirating 11.0 uL from A3 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Air gap
Aspirating 9.0 uL from A3 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 11.0 uL into G12 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at G12 of Corning 96 Well Plate 360 µL Flat on 6
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from G12 of Corning 96 Well Plate 360 µL Flat on 6 at 100.0 uL/sec
Dispensing 20.0 uL into G12 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 20.0 uL from G12 of Corning 96 Well Plate 360 µL Flat on 6 at 100.0 uL/sec
Dispensing 20.0 uL into G12 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from A6 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A6 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Aspirating 20.0 uL fro

Aspirating 20.0 uL from A5 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A5 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Blowing out at A5 of Agilent 6 Reservoir 47000 µL on 5
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from A4 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A4 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Aspirating 20.0 uL from A4 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A4 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Blowing out at A4 of Agilent 6 Reservoir 47000 µL on 5
Aspirating 7.0 uL from A4 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Air gap
Aspirating 13.0 uL from A4 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 7.0 uL into E6 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at E6 of Corning 96 Well Plate 360 µL Flat on 6
Mixing 2 times with a volume of 

Dispensing 100.0 uL into A6 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Blowing out at A6 of Agilent 6 Reservoir 47000 µL on 5
Mixing 2 times with a volume of 100.0 ul
Aspirating 100.0 uL from A5 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A5 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Aspirating 100.0 uL from A5 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A5 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Blowing out at A5 of Agilent 6 Reservoir 47000 µL on 5
Mixing 2 times with a volume of 100.0 ul
Aspirating 100.0 uL from A4 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A4 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Aspirating 100.0 uL from A4 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A4 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Blowing out at A4 of Agilent 6 Reservoir 47000 µL on 5
Aspirating 23.0 uL from 

Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from A6 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A6 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Aspirating 20.0 uL from A6 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A6 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Blowing out at A6 of Agilent 6 Reservoir 47000 µL on 5
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from A5 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A5 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Aspirating 20.0 uL from A5 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A5 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec
Blowing out at A5 of Agilent 6 Reservoir 47000 µL on 5
Mixing 2 times with a volume of 20.0 ul
Aspirating 20.0 uL from A4 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 20.0 uL into A4 of Agilent 6 Reserv

Dispensing 100.0 uL into E2 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Mixing 2 times with a volume of 100.0 ul
Aspirating 100.0 uL from A6 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A6 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Aspirating 100.0 uL from A6 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A6 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Blowing out at A6 of Agilent 6 Reservoir 47000 µL on 5
Mixing 2 times with a volume of 100.0 ul
Aspirating 100.0 uL from A5 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A5 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Aspirating 100.0 uL from A5 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A5 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Blowing out at A5 of Agilent 6 Reservoir 47000 µL on 5
Mixing 2 times with a volume of 100.0 ul
Aspirating 100.0 uL from A4 of Ag

Mixing 2 times with a volume of 100.0 ul
Aspirating 100.0 uL from F8 of Corning 96 Well Plate 360 µL Flat on 6 at 200.0 uL/sec
Dispensing 100.0 uL into F8 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 100.0 uL from F8 of Corning 96 Well Plate 360 µL Flat on 6 at 200.0 uL/sec
Dispensing 100.0 uL into F8 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Mixing 2 times with a volume of 100.0 ul
Aspirating 100.0 uL from A6 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A6 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Aspirating 100.0 uL from A6 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A6 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Blowing out at A6 of Agilent 6 Reservoir 47000 µL on 5
Mixing 2 times with a volume of 100.0 ul
Aspirating 100.0 uL from A5 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A5 of Agilent 6 Reservoir 47000 µL on 5 at 700.0

Aspirating 49.0 uL from B2 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Air gap
Aspirating 20.0 uL from B2 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 200.0 uL/sec
Dispensing 49.0 uL into H2 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at H2 of Corning 96 Well Plate 360 µL Flat on 6
Mixing 2 times with a volume of 100.0 ul
Aspirating 100.0 uL from H2 of Corning 96 Well Plate 360 µL Flat on 6 at 200.0 uL/sec
Dispensing 100.0 uL into H2 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 100.0 uL from H2 of Corning 96 Well Plate 360 µL Flat on 6 at 200.0 uL/sec
Dispensing 100.0 uL into H2 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Mixing 2 times with a volume of 100.0 ul
Aspirating 100.0 uL from A6 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 uL/sec
Dispensing 100.0 uL into A6 of Agilent 6 Reservoir 47000 µL on 5 at 700.0 uL/sec
Aspirating 100.0 uL from A6 of Agilent 6 Reservoir 47000 µL on 5 at 200.0 u

In [31]:
Pacific = timezone('US/Pacific')
p_time = datetime.now(Pacific)
print(p_time.strftime('%H:%M:%S'))

14:08:43
